# Portfolio Workflow

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import trading_factors_yahoo as alpha_factors
importlib.reload(alpha_factors)
import utils
importlib.reload(utils)
import nonoverlapping_estimator as ai_estimator
importlib.reload(ai_estimator)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')
print()

Sci-Kit version: 0.24.1
Sci-Kit version: 0.24.1
Python version: 3.8.8
Pandas version: 1.4.0



In [2]:
# For Chromedriver
from pathlib import Path

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = '../data/price_histories_yahoo.csv'

# Stage 1: Generate Stock Universe

- Gather stocks from specific criteria (SP500 top 50...)
- Use stock sentiment to select stocks
- Gather price histories

## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

# Price History data

In [3]:
number_of_years = 5
price_histories = pd.read_csv(price_histories_file_name, header=[0, 1], index_col=[0], parse_dates=True, low_memory=False)
print(f'Date range for price histories:         {price_histories.index.min()} to {price_histories.index.max()}')
latest_date = price_histories.index.max() 
earliest_date = latest_date - pd.DateOffset(years=number_of_years)
price_histories = price_histories[(price_histories.index >= earliest_date) & (price_histories.index <= latest_date)]
print(f'Working date range for price histories: {price_histories.index.min()} to {price_histories.index.max()}')
close = price_histories.Close
print(f'Date range for working close prices:    {close.index.min()} to {close.index.max()}')
print(f'You have {len(close.columns)} stocks')

Date range for price histories:         2017-04-20 00:00:00 to 2022-04-18 00:00:00
Working date range for price histories: 2017-04-20 00:00:00 to 2022-04-18 00:00:00
Date range for working close prices:    2017-04-20 00:00:00 to 2022-04-18 00:00:00
You have 424 stocks


In [4]:
price_histories

Attributes Adj_Ratio                                                         \
Symbols          AAL       AAP      ABBV       ABC       ABT       ACN ADBE   
Date                                                                          
2017-04-20  0.968483  0.965229  0.795611  0.917661  0.919901  0.928828  1.0   
2017-04-21  0.968482  0.965229  0.795611  0.917661  0.919901  0.928828  1.0   
2017-04-24  0.968482  0.965230  0.795611  0.917661  0.919901  0.928828  1.0   
2017-04-25  0.968482  0.965230  0.795611  0.917661  0.919901  0.928828  1.0   
2017-04-26  0.968483  0.965229  0.795611  0.917661  0.919901  0.928828  1.0   
...              ...       ...       ...       ...       ...       ...  ...   
2022-04-11  1.000000  1.000000  0.991573  1.000000  0.996030  0.997031  1.0   
2022-04-12  1.000000  1.000000  0.991573  1.000000  0.996030  0.997031  1.0   
2022-04-13  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-04-14  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-04-18  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   

Attributes                                ...    Volume                       \
Symbols          ADI       ADM       ADP  ...       WST       WTW         WY   
Date                                      ...                                  
2017-04-20  0.906076  0.860876  0.899866  ...  339300.0  488800.0  4268800.0   
2017-04-21  0.906076  0.860876  0.899866  ...  306700.0  798100.0  4338800.0   
2017-04-24  0.906076  0.860876  0.899866  ...  232100.0  859900.0  5779400.0   
2017-04-25  0.906076  0.860876  0.899866  ...  438300.0  505900.0  7561600.0   
2017-04-26  0.906076  0.860876  0.899866  ...  626300.0  819500.0  8544700.0   
...              ...       ...       ...  ...       ...       ...        ...   
2022-04-11  1.000000  1.000000  1.000000  ...  311000.0  535000.0  3780100.0   
2022-04-12  1.000000  1.000000  1.000000  ...  288000.0  725100.0  3150600.0   
2022-04-13  1.000000  1.000000  1.000000  ...  509000.0  742100.0  3740600.0   
2022-04-14  1.000000  1.000000  1.000000  ...  258000.0  392900.0  3334200.0   
2022-04-18  1.000000  1.000000  1.000000  ...  308300.0  277300.0  3581300.0   

Attributes                                                                    \
Symbols           XEL         XOM       XRAY        XYL        YUM      ZBRA   
Date                                                                           
2017-04-20  2919900.0  11649100.0  1307500.0  1191300.0  2448100.0  271500.0   
2017-04-21  1937700.0   9816700.0  1381800.0  1499700.0  2595700.0  214800.0   
2017-04-24  2264800.0   8729700.0  1140300.0  1796500.0  2525500.0  304500.0   
2017-04-25  2027500.0  10733800.0  1097000.0  1366400.0  3411300.0  358000.0   
2017-04-26  2135600.0   9445900.0   883800.0  1197400.0  1684300.0  245600.0   
...               ...         ...        ...        ...        ...       ...   
2022-04-11  2752300.0  21800100.0  1728600.0  1076700.0  2199600.0  249100.0   
2022-04-12  2872300.0  22444200.0  1266300.0  1172900.0  2396300.0  276100.0   
2022-04-13  2274800.0  19306800.0  1213400.0  1061500.0  1710900.0  466200.0   
2022-04-14  2007100.0  24544700.0   887800.0  3518600.0  1830700.0  268400.0   
2022-04-18  2180900.0  19013400.0  1254500.0  1337300.0  2731500.0  262000.0   

Attributes             
Symbols           ZTS  
Date                   
2017-04-20  2232800.0  
2017-04-21  2124700.0  
2017-04-24  2770600.0  
2017-04-25  4346100.0  
2017-04-26  2665800.0  
...               ...  
2022-04-11  2252200.0  
2022-04-12  1746500.0  
2022-04-13  1517800.0  
2022-04-14  1819400.0  
2022-04-18  1857000.0  

[1258 rows x 2544 columns]

In [5]:
close

Symbols,AAL,AAP,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,...,WST,WTW,WY,XEL,XOM,XRAY,XYL,YUM,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2017-04-20,43.794781,141.464020,50.744049,75.587700,40.457249,110.725624,131.419998,71.271957,38.653324,92.722214,...,80.307701,129.860001,28.165520,39.493187,62.264553,60.591255,47.321918,59.914917,90.820000,52.772923
2017-04-21,43.726982,141.734253,50.775867,74.569122,40.043282,110.744194,131.520004,70.338692,38.575840,92.407257,...,80.337120,128.050003,27.987364,39.652397,62.018589,60.446259,47.727097,59.712246,90.970001,52.675709
2017-04-24,44.976322,141.985275,51.444183,74.275452,40.595230,110.864937,132.889999,71.779358,39.066544,93.712059,...,80.866821,130.080002,28.254597,39.820461,62.341404,61.219604,48.499786,60.541328,93.019997,53.453342
2017-04-25,45.140968,140.604996,51.794247,74.780174,40.254868,110.242607,133.490005,71.897133,39.238712,94.494965,...,81.288620,131.789993,27.533869,39.811615,62.817928,60.881268,48.820160,60.780838,95.040001,53.579723
2017-04-26,44.937595,140.711121,51.746510,74.559921,40.291668,110.511986,132.789993,70.655846,38.971844,94.359962,...,82.397049,132.529999,27.744411,39.661251,62.564293,61.190605,48.480938,60.688705,93.839996,54.017151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-11,16.969999,220.970001,168.398773,162.649994,119.563454,326.298401,434.440002,154.889999,94.570000,230.960007,...,398.500000,238.279999,39.020000,74.360001,83.849998,48.480000,85.730003,118.309998,408.700012,192.500000
2022-04-12,17.129999,221.880005,165.900009,162.619995,117.920006,325.760010,426.769989,155.500000,95.070000,231.059998,...,384.549988,232.460007,39.580002,74.730003,85.599998,48.450001,84.849998,119.889999,402.209991,187.910004
2022-04-13,18.950001,220.580002,158.949997,162.960007,119.480003,325.589996,431.670013,158.119995,95.550003,233.740005,...,387.019989,233.880005,40.060001,74.370003,86.809998,48.750000,86.540001,121.870003,405.000000,189.520004


# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [6]:
snp_500_stocks = utils.get_snp500()
sector_helper = alpha_factors.get_sector_helper(snp_500_stocks, 'GICS Sector', close.columns)
all_factors = pd.concat(
[
    alpha_factors.FactorMomentum(price_histories, 252).demean(groupby=sector_helper.values()).rank().zscore().for_al(),
    alpha_factors.TrailingOvernightReturns(price_histories, 10).rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 120).demean(groupby=sector_helper.values()).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al()
], axis=1)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

# Stage 2b: Generate AI Alpha Factors

- Compute target values (y)
    - Quantize with 2 bins
- Train model for Feature importance
- Feature reduction
- Train model for AI Alpha Vector
- Compute AI Alpha Vectors for 1 year
- Save AI Alpha Vectors

## Compute the target values (y) and Shift back to create a 5 day forward prediciton

This is something you want to experiment with. If you are planning on holding on to assets for long periods of time, perhaps a 20, 40 or 60 forward prediciton will work better.

In [7]:
prod_target_source = '5Day2Quant'
prod_target_quantiles = 2
forward_prediciton_days = 5

all_assets = all_factors.index.levels[1].values.tolist()
print(f'Factors from date: {all_factors.index.levels[0].min()} to date: {all_factors.index.levels[0].max()}')
features = all_factors.columns.tolist()

all_factors = pd.concat(
[
    all_factors,
    alpha_factors.FactorReturnQuantiles(price_histories, prod_target_quantiles, forward_prediciton_days).for_al(prod_target_source),
], axis=1).dropna()
all_factors.sort_index(inplace=True)

all_factors['target'] = all_factors.groupby(level=1)[prod_target_source].shift(-forward_prediciton_days)

Factors from date: 2017-05-03 00:00:00 to date: 2022-04-18 00:00:00


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
n_days = 10
n_stocks = len(set(all_factors.index.get_level_values(level=1).values))
clf_random_state = 42

clf_parameters = {
    'criterion': 'entropy',
    'min_samples_leaf': n_days * n_stocks,
    'oob_score': True,
    'n_jobs': -1,
    'random_state': clf_random_state}
n_trees_l = [50, 100, 250, 1000]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

temp = all_factors.dropna().copy()
X = temp[features]
y = temp['target']

clf = RandomForestClassifier(1000, **clf_parameters)

clf_nov = ai_estimator.NoOverlapVoter(clf, n_skip_samples=forward_prediciton_days-1)
clf_nov.fit(X, y)

print(f'train: {clf_nov.score(X, y.values)} oob: {clf_nov.oob_score_}')

In [ ]:
ai_alpha_name = 'AI_ALPHA'
factors_with_alpha = alpha_factors.add_alpha_score(all_factors[features].copy(), clf_nov, ai_alpha_name)

In [ ]:
factors_with_alpha

In [ ]:
factors_to_compare = ['trailing_overnight_returns_10_day_smoothed', 'mean_reversion_120_day_logret', 'annualzed_volatility_120_day', 'average_dollar_volume_120_day']
alpha_factors.evaluate_ai_alpha(factors_with_alpha[factors_to_compare + [ai_alpha_name]], close)

In [ ]:
ai_alpha = factors_with_alpha[ai_alpha_name].copy()
alpha_vectors = ai_alpha.reset_index().pivot(index='Date', columns='Symbols', values=ai_alpha_name)
alpha_vectors.reset_index().to_csv('data/alpha_vectors.csv', index=False)

# Stage 3: Generate Beta Factors

- Use Risk Model
- Compute Daily Betas for 1 year

In [ ]:
returns = alpha_factors.FactorReturns(price_histories).factor_data.dropna()
end_date = returns.index.max()
start_date = end_date - pd.offsets.DateOffset(years=number_of_years - 1)
print(f'Generating {number_of_years - 1} year Betas from {start_date} to {end_date}')
beta_dates = pd.date_range(start_date, end_date, freq='D')
daily_betas = {}
for beta_date in returns[start_date:].index:
    start_of_returns = beta_date - pd.offsets.DateOffset(years=1)
    beta_returns = returns.loc[start_of_returns:beta_date]
    risk_model = alpha_factors.RiskModelPCA(beta_returns, 1, 20)
    daily_betas[beta_date.strftime('%m/%d/%Y')] = risk_model

In [ ]:
with open('data/daily_beta.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(daily_betas, f, pickle.HIGHEST_PROTOCOL)

# Stage 4a: Demonstrate using AI Alpha and Daily Betas to produce optimal weights

In [ ]:
price_histories = pd.read_csv(price_histories_file_name, header=[0, 1], index_col=[0], parse_dates=True, low_memory=False)
print(f'Date range for price histories: {price_histories.index.min()} to {price_histories.index.max()}')
pricing = price_histories.Close
print(f'You have {len(pricing.columns)} stocks from picing')

alpha_vectors = pd.read_csv('data/alpha_vectors.csv', parse_dates=['Date']).set_index(['Date']).sort_index()
print(f'You have {len(alpha_vectors.columns)} stocks from alpha')

with open('data/daily_beta.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    daily_betas = pickle.load(f)
print(f'You have {len(daily_betas)} of daily betas')

## Generate Daily Optimal Portfolios using 1 year of alpha and beta

This first strategy is to use 2 day returns and optimize the portfolio daily.

The second stragety is to use last day of the month returns and to optimize the portfolio monthly (or 20 day returns).

Start with the last date, subtract 1 year to get the start and end dates for the betas.

In [ ]:
import portfolio_optimizer
from portfolio_optimizer import OptimalHoldings
importlib.reload(portfolio_optimizer)

risk_cap = 0.15
weights_max = 0.25
weights_min = 0.00

## 5 Day Strategy Backtest

Use 5 Day returns and optimize portfolio weekly. 

We are looking for something at 8% return or better

In [ ]:
returns = alpha_factors.FactorReturns(price_histories).factor_data
dlyreturn_n_days_delay = 5
delayed_returns = returns[-252:].shift(-dlyreturn_n_days_delay).dropna()
start_date = list(delayed_returns.index)[0]
end_date = list(delayed_returns.index)[-1]
print(f'Generating 1 year Optimal Portfolios from {start_date} to {end_date}')
current_holdings = pd.DataFrame(np.zeros(len(delayed_returns.columns)), index=delayed_returns.columns)
init_port_value = portfolio_value = 100000
portfolio_growth = {}
for opt_date in tqdm(delayed_returns.index.to_list()[-252::dlyreturn_n_days_delay], desc='Dates', unit='Portfolio Optimization'):
    alpha_vector = pd.DataFrame(alpha_vectors.loc[opt_date])
    risk_model = daily_betas[opt_date.strftime('%m/%d/%Y')]
    est_return = delayed_returns.loc[opt_date]
    optimal_weights = OptimalHoldings(risk_cap=risk_cap,weights_max=weights_max, weights_min=weights_min).find(alpha_vector, risk_model.factor_betas_, risk_model.factor_cov_matrix_, risk_model.idiosyncratic_var_vector_)
    long_weights = optimal_weights[(100 * optimal_weights['optimalWeights']).round() > 0]
    long_holdings = (long_weights['optimalWeights'] * portfolio_value).round(0)
    new_holdings = long_holdings + (long_holdings * est_return[long_holdings.index])
    portfolio_value = new_holdings.sum()
    portfolio_growth[opt_date] = portfolio_value
    current_holdings = new_holdings

In [ ]:
pd.Series(portfolio_growth).plot()

In [ ]:
port_return = round(np.log(portfolio_value / init_port_value) * 100, 2)
print(f'Starting portfolio: {init_port_value} Ending portfolio: {portfolio_value} Return: {port_return}%')
if port_return >= 8:
    print('Backtest indicates its okay to proceed with this strategy.')
else:
    raise RuntimeError(f'Backtest indicates this strategy needs more work! ({port_return})') from None

In [ ]:
pd.Series(portfolio_growth).plot()

## Use the last week to determine current portfolio mix

In [ ]:
opt_date = alpha_vectors.index[-1]
print(f'From date: {opt_date}')
risk_model = daily_betas[opt_date.strftime('%m/%d/%Y')]
alpha_vector = pd.DataFrame(alpha_vectors.loc[opt_date])
optimal_weights = OptimalHoldings(risk_cap=risk_cap,weights_max=weights_max, weights_min=weights_min).find(alpha_vector, risk_model.factor_betas_, risk_model.factor_cov_matrix_, risk_model.idiosyncratic_var_vector_)

In [ ]:
long_weights = optimal_weights[(100 * optimal_weights['optimalWeights']).round() > 5.0]
returns[-252:][long_weights.index.to_list()].cumsum().plot()
print(f'New portfolio variance is:  {risk_model.compute_portfolio_variance(optimal_weights):.8f}')

In [ ]:
long_weights

In [ ]:
td_ameritrade = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')
td_ameritrade.authenticate()
td_ameritrade.get_fundamental(list(long_weights.index.to_list()))

In [ ]:
parsed_and_scored_news = utils.get_finvis_stock_sentiment(long_weights.index.to_list()).sort_values(by='date')
# Group by date and ticker columns from scored_news and calculate the mean
mean_scores = parsed_and_scored_news.groupby(['ticker','date']).mean()
# Unstack the column ticker
mean_scores = mean_scores.unstack()
# Get the cross-section of compound in the 'columns' axis
mean_scores = mean_scores.xs('compound', axis="columns").transpose()
# Plot a bar chart with pandas
mean_scores[-20:].plot(kind = 'bar')
plt.grid()

## First lets see which stocks we already own for a specific account

I only want to work with Equity investments. This is kind of confusing, but at the account level assets that can be traded are call "EQUITY". When you get quotes for each asset, the same asset can be something like "ETF".

I also use Ameritrade's portfolio planner tool to create an asset mix based off of their reccomendations. I don't want these stocks (or in my case mutual funds and ETFs) to be part of this analysis. So I'll remove them here.

## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

## Authentication Tokens

To get data from Ameritrade you will need to obtains a short time use token (there is a re-use token, but I have not coded it yet.) You only need to do this if you
are going to use an existing Ameritrade account to define an initial set of stocks to analyze.

To obtain a token, you will need to have a Chrome driver located somewhere on your system. This will allow the module to use your credentials to obtain an authentication token.

For security reasons, I sugges using environment variables to store your credential information. If you store them in property files, or just code them into your notebook, you risk sharing the information with others if you use GitHub or some other SCCS. This also makes it easier to have them availabe from project to project in your development environment

<span style="color:blue">Note: *Account numbers are masked for security purposes.*</span>

In [ ]:
td_ameritrade = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')
td_ameritrade_ameritrade.authenticate()

if len(td_ameritrade.authorization) == 0:
    print('Error: No authorization data: {}'.format(td_ameritrade.authorization))
else:
    print('You have authorization')

print(f'Date of trade: {datetime.today()}')

# Specific Portfolio Account
account_portfolio_df = utils.get_account_portfolio_data(td_ameritrade.parse_portfolios_list(), masked_account_number)
equity_investments_df = utils.get_investments_by_type(account_portfolio_df, investment_type='EQUITY')
print('Full Equity Portfolio:')
display(equity_investments_df)

long_term_stocks =  ['FGPHF', 'WKHS', 'EEENF']

# Filter out non Equity investments
current_stocks = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid').get_quotes(
    utils.get_investment_symbols(equity_investments_df)).query('assetType == "EQUITY"').index.tolist()
stocks_to_sell = equity_investments_df[~equity_investments_df['symbol'].isin(long_term_stocks)]
print('Stocks to sell:')
stocks_to_sell

### Submit and montior sell orders

In [ ]:
td_ameritrade = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')
td_ameritrade.authenticate()
td_ameritrade.parse_portfolios_list()
account_number = td_ameritrade.unmask_account(masked_account_number)
sell_result = td_ameritrade.place_bulk_sell_orders(account_number, stocks_to_sell)

In [ ]:
while True:
    td_ameritrade = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')
    td_ameritrade.authenticate()
    account_portfolio_df = utils.get_account_portfolio_data(td_ameritrade.parse_portfolios_list(), masked_account_number)
    equity_investments_df = utils.get_investments_by_type(account_portfolio_df, investment_type='EQUITY')
    stocks_with_sell_orders = equity_investments_df[~equity_investments_df['symbol'].isin(long_term_stocks)]
    if len(stocks_with_sell_orders[stocks_with_sell_orders.longQuantity >= 1]) == 0:
        break
    print(f'\r{datetime.today()} - Stocks to sell: {stocks_with_sell_orders.symbol.values}', end='           ')
    time.sleep(60)
    
print(f'\r{datetime.today()} - All stocks sold: {stocks_to_sell.symbol.values}')

In [ ]:
account_balances = td_ameritrade.parse_accounts().loc[masked_account_number]
print(account_balances)
print(account_balances.currentBalances_cashBalance)
print(account_balances.currentBalances_moneyMarketFund)
available_cash = account_balances.currentBalances_cashBalance + account_balances.currentBalances_moneyMarketFund
print(f'Total cash to invest: {available_cash}')

In [ ]:
print(f'Available cash  : {available_cash}')
investment_base = 500
investment_amount = math.floor(available_cash / investment_base) * investment_base
print(f'Amount to invest: {investment_amount}')

In [ ]:
investment_amount = 5000

In [ ]:
long_weights

In [ ]:
print(datetime.today())
print(f'Initial investment amount: {investment_amount}')
nearest_base = 5
min_shares = 5
long_quotes = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid').get_quotes(long_weights.index.to_list())
long_shares = long_quotes['regularMarketLastPrice'].to_frame()
long_shares['optimalWeights'] = long_weights['optimalWeights']
long_shares['invest_amount'] = (long_shares['optimalWeights'] * investment_amount).round(0)
long_shares['shares'] = (long_shares['invest_amount'] / long_shares['regularMarketLastPrice']).astype(int)
display(long_shares)

# Remove symbols that are less than min and re-compute

final_long_shares = long_shares.query('shares >= @min_shares').copy()
final_long_shares['optimalWeights'] = final_long_shares['optimalWeights'] / final_long_shares['optimalWeights'].sum(axis=0)
final_long_shares['invest_amount'] = (final_long_shares['optimalWeights'] * investment_amount).round(0)
final_long_shares['shares'] = (final_long_shares['invest_amount'] / final_long_shares['regularMarketLastPrice']).astype(int)
final_long_shares['cost'] = final_long_shares['shares'] * final_long_shares['regularMarketLastPrice']
display(final_long_shares)
print(f'Total cost: {final_long_shares.cost.sum()}')


### Place buy orders

In [ ]:
amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid').get_quotes(long_weights.index.to_list())
td_ameritrade.get_fundamental(list(final_long_shares.index))

In [ ]:
print(datetime.today())
td_ameritrade = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')
td_ameritrade.authenticate()
td_ameritrade.parse_portfolios_list()
account_number = td_ameritrade.unmask_account(masked_account_number)

for index, row in final_long_shares.iterrows():
    fundamental = td_ameritrade.get_fundamental([index])
    print(f'Placing BUY order on {account_number} for {row.shares} shares of {index}:{fundamental.loc[0].assetType}...')
    result = td_ameritrade.place_order(account_number, index, fundamental.loc[0].assetType, row.shares, 'BUY')

In [ ]:
portfolio_price_histories = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid').get_price_histories(list(long_shares.index), datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)
portfolio_close = utils.get_close_values(portfolio_price_histories)
utils.compute_log_returns(portfolio_close)[-2:].cumsum().plot()

In [ ]:
import ameritrade_functions as amc
importlib.reload(amc)

td_ameritrade = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')
td_ameritrade.authenticate()
td_ameritrade.parse_accounts().loc[masked_account_number]